# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 24 2022 7:36AM,237925,19,ADV80002838,"AdvaGen Pharma, Ltd",Released
1,May 24 2022 7:36AM,237925,19,ADV80002840,"AdvaGen Pharma, Ltd",Released
2,May 24 2022 7:36AM,237925,19,ADV80002843,"AdvaGen Pharma, Ltd",Released
3,May 24 2022 7:36AM,237925,19,ADV80002844,"AdvaGen Pharma, Ltd",Released
4,May 24 2022 7:36AM,237925,19,ADV80002845,"AdvaGen Pharma, Ltd",Released
5,May 24 2022 7:36AM,237925,19,ADV80002846,"AdvaGen Pharma, Ltd",Released
6,May 23 2022 4:46PM,237924,17,8053762,Twinlab Consolidated Corporation,Released
7,May 23 2022 4:25PM,237893,16,8001704,Sartorius Bioprocess Solutions,Released
8,May 23 2022 4:25PM,237893,16,8002225,Sartorius Bioprocess Solutions,Released
9,May 23 2022 4:25PM,237893,16,8036524,Sartorius Bioprocess Solutions,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,237916,Executing,5
29,237916,Released,37
30,237922,Released,1
31,237924,Released,1
32,237925,Released,6


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237914,NaN,NaN,20.0
237916,NaN,5.0,37.0
237922,NaN,NaN,1.0
237924,NaN,NaN,1.0
237925,NaN,NaN,6.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237673,0.0,0.0,1.0
237701,0.0,0.0,1.0
237808,0.0,0.0,1.0
237813,0.0,0.0,3.0
237815,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237673,0,0,1
237701,0,0,1
237808,0,0,1
237813,0,0,3
237815,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237673,0,0,1
1,237701,0,0,1
2,237808,0,0,1
3,237813,0,0,3
4,237815,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237673,,,1
1,237701,,,1
2,237808,,,1
3,237813,,,3
4,237815,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 24 2022 7:36AM,237925,19,"AdvaGen Pharma, Ltd"
6,May 23 2022 4:46PM,237924,17,Twinlab Consolidated Corporation
7,May 23 2022 4:25PM,237893,16,Sartorius Bioprocess Solutions
17,May 23 2022 4:10PM,237922,16,"SHL Pharma, LLC"
18,May 23 2022 3:25PM,237916,20,Alumier Labs Inc.
60,May 23 2022 3:12PM,237914,20,"Exact-Rx, Inc."
80,May 23 2022 3:08PM,237912,10,"Methapharm, Inc."
82,May 23 2022 2:35PM,237701,18,Hush Hush
83,May 23 2022 2:27PM,237897,20,Sartorius Stedim North America
84,May 23 2022 2:24PM,237896,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 24 2022 7:36AM,237925,19,"AdvaGen Pharma, Ltd",,,6
1,May 23 2022 4:46PM,237924,17,Twinlab Consolidated Corporation,,,1
2,May 23 2022 4:25PM,237893,16,Sartorius Bioprocess Solutions,,,10
3,May 23 2022 4:10PM,237922,16,"SHL Pharma, LLC",,,1
4,May 23 2022 3:25PM,237916,20,Alumier Labs Inc.,,5,37
5,May 23 2022 3:12PM,237914,20,"Exact-Rx, Inc.",,,20
6,May 23 2022 3:08PM,237912,10,"Methapharm, Inc.",,,2
7,May 23 2022 2:35PM,237701,18,Hush Hush,,,1
8,May 23 2022 2:27PM,237897,20,Sartorius Stedim North America,,,1
9,May 23 2022 2:24PM,237896,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 24 2022 7:36AM,237925,19,"AdvaGen Pharma, Ltd",6,,
1,May 23 2022 4:46PM,237924,17,Twinlab Consolidated Corporation,1,,
2,May 23 2022 4:25PM,237893,16,Sartorius Bioprocess Solutions,10,,
3,May 23 2022 4:10PM,237922,16,"SHL Pharma, LLC",1,,
4,May 23 2022 3:25PM,237916,20,Alumier Labs Inc.,37,5,
5,May 23 2022 3:12PM,237914,20,"Exact-Rx, Inc.",20,,
6,May 23 2022 3:08PM,237912,10,"Methapharm, Inc.",2,,
7,May 23 2022 2:35PM,237701,18,Hush Hush,1,,
8,May 23 2022 2:27PM,237897,20,Sartorius Stedim North America,1,,
9,May 23 2022 2:24PM,237896,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 24 2022 7:36AM,237925,19,"AdvaGen Pharma, Ltd",6,,
1,May 23 2022 4:46PM,237924,17,Twinlab Consolidated Corporation,1,,
2,May 23 2022 4:25PM,237893,16,Sartorius Bioprocess Solutions,10,,
3,May 23 2022 4:10PM,237922,16,"SHL Pharma, LLC",1,,
4,May 23 2022 3:25PM,237916,20,Alumier Labs Inc.,37,5,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 24 2022 7:36AM,237925,19,"AdvaGen Pharma, Ltd",6.0,NaN,NaN
1,May 23 2022 4:46PM,237924,17,Twinlab Consolidated Corporation,1.0,NaN,NaN
2,May 23 2022 4:25PM,237893,16,Sartorius Bioprocess Solutions,10.0,NaN,NaN
3,May 23 2022 4:10PM,237922,16,"SHL Pharma, LLC",1.0,NaN,NaN
4,May 23 2022 3:25PM,237916,20,Alumier Labs Inc.,37.0,5.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 24 2022 7:36AM,237925,19,"AdvaGen Pharma, Ltd",6.0,0.0,0.0
1,May 23 2022 4:46PM,237924,17,Twinlab Consolidated Corporation,1.0,0.0,0.0
2,May 23 2022 4:25PM,237893,16,Sartorius Bioprocess Solutions,10.0,0.0,0.0
3,May 23 2022 4:10PM,237922,16,"SHL Pharma, LLC",1.0,0.0,0.0
4,May 23 2022 3:25PM,237916,20,Alumier Labs Inc.,37.0,5.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,713605,7.0,0.0,0.0
12,475723,1.0,1.0,0.0
16,713702,11.0,1.0,0.0
17,237924,1.0,0.0,0.0
18,1188822,5.0,0.0,0.0
19,951570,11.0,1.0,17.0
20,1665248,104.0,5.0,0.0
21,475791,2.0,0.0,0.0
22,475773,0.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,713605,7.0,0.0,0.0
1,12,475723,1.0,1.0,0.0
2,16,713702,11.0,1.0,0.0
3,17,237924,1.0,0.0,0.0
4,18,1188822,5.0,0.0,0.0
5,19,951570,11.0,1.0,17.0
6,20,1665248,104.0,5.0,0.0
7,21,475791,2.0,0.0,0.0
8,22,475773,0.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,7.0,0.0,0.0
1,12,1.0,1.0,0.0
2,16,11.0,1.0,0.0
3,17,1.0,0.0,0.0
4,18,5.0,0.0,0.0
5,19,11.0,1.0,17.0
6,20,104.0,5.0,0.0
7,21,2.0,0.0,0.0
8,22,0.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,7.0
1,12,Released,1.0
2,16,Released,11.0
3,17,Released,1.0
4,18,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,17,18,19,20,21,22
Status,,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0
Executing,0.0,1.0,1.0,0.0,0.0,1.0,5.0,0.0,2.0
Released,7.0,1.0,11.0,1.0,5.0,11.0,104.0,2.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,17,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0
1,Executing,0.0,1.0,1.0,0.0,0.0,1.0,5.0,0.0,2.0
2,Released,7.0,1.0,11.0,1.0,5.0,11.0,104.0,2.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,17,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0
1,Executing,0.0,1.0,1.0,0.0,0.0,1.0,5.0,0.0,2.0
2,Released,7.0,1.0,11.0,1.0,5.0,11.0,104.0,2.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()